# Preguntas y respuestas (Consultas ejecutables)

# ENTREGABLE 3

## Nota! Ejecutar primera celda para disponibilizar datos e los sv que se usan en las posteriores cnsultas ! 


In [1]:
import pandas as pd
sales_df = pd.read_csv(r'data/sales.csv')
products_df = pd.read_csv(r'data/products.csv')

## Columna con datos inválidos
__Issue__ - El campo TotalPrice en la tabla sales no tiene valores válidos. Utilizando la información de precios de la tabla products, calcula el valor real de la venta para cada registro y almacena en una nueva columna
        - Utiliza la siguiente fórmula: TotalPriceCalculated=(Quantity×UnitPrice)×(1−Discount)

__Análisis__

Consulta cuántos registros n totalPrice son menor/ igual a cero. Y por las dudas si discount se escapa del rango (para validar la formula de correcio )

In [ ]:
import pandas as pd
import utils.pandas_utils as p_utils

import utils.notebook_utils as notebook_utils

p_utils.get_dataframe_info(sales_df)

notebook_utils.print_colored('Total rows on TotalPrice (zero or negative):', 'orange')
print(p_utils.sum_zero_or_negative(sales_df, 'TotalPrice'))

notebook_utils.print_colored('Total rows on Discount (not between 0 and 1):', 'orange')
print(len(sales_df['Discount'].between(0, 1, inclusive='both')))

__Solución__

 Se procede a corregir

In [2]:
import utils.sql_utils as sql_utils
import utils.notebook_utils as notebook_utils
import utils.pandas_utils as p_utils

sales_data= sales_df.copy()
products_data = products_df.copy()

sales_with_price = sales_data.merge(products_data[['ProductID', 'Price']], on='ProductID', how='left') # agrega el precio del producto

sales_with_price['AggregatedTotalPrice'] = sales_with_price['Quantity'] * sales_with_price['Price'] * (1 - sales_with_price['Discount'])
sales_with_price.head()

notebook_utils.print_colored('Total rows on AggregatedTotalPrice (zero or negative):', 'green')
print(p_utils.sum_zero_or_negative(sales_with_price, 'AggregatedTotalPrice'))


45353


__Comentario__

aun quedan filas en cer, dado el calculo que hacemos tiene que ser quantity = 0  o Price = 0 y se comprueba que existen dos productos con precio = 0 (ID 19 y 155)

In [ ]:
# buscar los productos con precio cero o negativo y obtener filas en la tabla de ventas
zero_or_negative_products = products_data[products_data['Price'] <= 0]
notebook_utils.print_colored('Total rows on products with zero or negative price:', 'red')
print(zero_or_negative_products)


## Detecta los outliers en la columna de ventas totales (TotalPriceCalculated)

__Issue__ - Utilizando el criterio del rango intercuartílico (IQR). Luego, crea una nueva columna llamada IsOutlier que tenga el valor 1 si el registro es un outlier y 0 en caso contrario. ¿Cuántos outliers se detectaron?

__Análisis__

Consulta cuántos registros n totalPrice son menor/ igual a cero. Y por las dudas si discount se escapa del rango (para validar la formula de correcio )

In [ ]:
 # calclular q1,q2,q3 y el rango intercuartil
q1 = sales_with_price['AggregatedTotalPrice'].quantile(0.25)
q2 = sales_with_price['AggregatedTotalPrice'].quantile(0.5)
q3 = sales_with_price['AggregatedTotalPrice'].quantile(0.75)

iqr = q3 - q1
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr

notebook_utils.print_colored('Quartiles and IQR of AggregatedTotalPrice:', 'blue')
notebook_utils.print_colored(f'IQR: ${iqr}', 'violet')
notebook_utils.print_colored(f'Q2: ${q2}', 'violet')
notebook_utils.print_colored(f'Q3: ${q3}', 'violet')
notebook_utils.print_colored(f'Lower Bound: ${lower_bound}', 'violet')
notebook_utils.print_colored(f'Upper Bound: ${upper_bound}', 'violet')

sales_data = sales_with_price.copy()

sales_data['isOutlier'] = sales_data['AggregatedTotalPrice'].apply(
    lambda x: x < lower_bound or x > upper_bound
).astype(int)

notebook_utils.print_colored('Total rows on AggregatedTotalPrice outliers:', 'purple')
print(sales_with_price['isOutlier'].sum())

sales_with_price.head()
